# Predicting survivols for the Titanic Dataset

## Introduction

The sinking of the RMS Titanic is one of the most infamous shipwrecks in history. On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

This project aims to examine these differences in survival probability in depth. We hope to create an accurate model for predicting individual level survival probabilities. We chose survival status as our dependent variable and age, passenger class, sex, and embarkment city to be our independent variables. We aim at learning new insights on how these features affected the survival probablity of the participants in this tragedy that shook the world. 

## Data

The dataset we worked on was ready-made and aquired from Vanderbilt Biostatistics dataset collection (http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic.html). The dataset relies on multiple sources major one being the Encyclopedia Titanica webpage. It contains information about all 1313 passengers including their whole names, sex, age, passenger class, and survival status among others. The dataset does not include information about the RMS Titanic crew. RMS Titanic had three different passenger classes.

### Preprocessing

The dataset required preprocesing before it could be used in modelling. First, categorical variables (sex, passenger class, embarkment city) were transformed into binary form. Second, all rows including missing values were dropped out. It is notable that this removes major part of the original data. Moreover, has different effect on different passenger classes as over 70% of 3rd class passenger data is lost. Third, age variable was scaled and normalized to avoid its larger magnitude to skew results. Finally, data set was divided into training (67 %) and test (33 %) set.

### Key Statistics

Key statistics were computed for the dataset. The table below presenst the key statistics. 
\begin{table}[H]
\centering
\caption{Key statistics by passenger class}
\resizebox{0.95\textwidth}{!}{
    \input{key_stats.tex}
}
\end{table}


## Model

Our choice is to examine the data with logistic regression. In logistic regression linear regression is nonlinearized with a logistic function. The function limits regression values between 0 and 1 and thus provides a prozy for probability of survival.

### Prior choice

As we don't have prior knowledge on how the data is distributed, a weakly informative prior is chosen according to stan development team guidelines: student_t(nu,0,s) distribution, where s is chosen to provide weak information on the expected scale, and 3<nu<7.

### Model Selection

This prior further adjusted in model selection. Several different combinations of degrees of freedom and scale values for prior were compared by applying PSIS-LOO cross validation to the logistic model. The table and the figure below present results of the cross validation. It can be seen that prior with 6 degrees of freedom and scale as 1 provides the best fit based on the PSIS-LOO value and corresponding k values.

\begin{table}[H]
\centering
\caption{PSIS-LOO cross validation results}
\resizebox{0.45\textwidth}{!}{
    \input{psis_loo_results.tex}
}
\end{table}

![PSIS-LOO k vales with different prior parameters](k_values.png)

### Sensitivity Analysis

### Convergence Diagnostic

Based on the model selection the final predictive model was fitted with t(6,0,1) as prior, see table below for convergence diagnostics. Based on the convergence diagnostic we the model has convergenced well (R_hat < 1.01)

\begin{table}[H]
\centering
\caption{Logistic regression converge statistics}
\resizebox{0.75\textwidth}{!}{
    \input{fit_logistic_regression.tex}
}
\end{table}

### Model Parameters
Betas of the regression model are presented in the figure below.
![Beta draws from posterior](betas.png)


## Results

Based on the beta values, the probability of survival is increased by being young and female and having a higher class ticket. The embarkment city seems not have any effect which is pretty intuitive. 

Training and test error are presented in a table below. We can see that the logistic regression model achieves a significat improvement to a dummy model which predicts a person not surviving

\begin{table}[H]
\centering
\caption{F1-scores}
\resizebox{0.75\textwidth}{!}{
    \input{f1_scores.tex}
}
\end{table}

## Conclusion

We were able train a classifier to predict survivols with 0.74 F1 score and found out the intuitive fact that being young, female and having higher class ticket improved individuals changes to survive in titanic

## Appendix

In [1]:
import os
import math
import sys
import pystan
import numpy as np
import pandas as pd
import itertools
import pickle

from IPython.display import display
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from matplotlib import image, pyplot as plt
from tqdm import tqdm
from sklearn.metrics import f1_score
from utilities import psis, stan_utility
from utilities.my_utilities import *
plt.style.use("ggplot")

In [2]:
CONVERT_TO_PDF = True
RUN_MODEL_SELECTION = False

In [3]:
# data
data = pd.read_csv("../data/titanic.txt", index_col="name").drop(["row.names", 
                                                                  "home.dest", 
                                                                  "room", 
                                                                  "ticket", 
                                                                  "boat"], axis=1)

### Key statistics

In [4]:
pclass_summary = data.groupby('pclass').agg({'survived': ['sum', 'mean'], 
                                             'age': [no_info, 'mean'], 
                                             'sex': [females, males, tot], 
                                             'embarked': [Queenstown, Cherbourg, Southampton, no_info, tot]
})
pclass_summary["tot"] = data.pclass.value_counts()

final = pd.DataFrame()
final["Passengers"] = pclass_summary["tot"]
final["Men"] = pclass_summary["sex"]["males"]
final["Women"] = pclass_summary["sex"]["females"]
final["Age (avg)"] = pclass_summary["age"]["mean"]
final["Age unknown"] = pclass_summary["age"]["no_info"]
final["Queenstown"] = pclass_summary["embarked"]["Queenstown"]
final["Cherbourg"] = pclass_summary["embarked"]["Cherbourg"]
final["Southampton"] = pclass_summary["embarked"]["Southampton"]
final["Survived"] = pclass_summary["survived"]["sum"]
final["Percentage"] = pclass_summary["survived"]["mean"]

create_tex_table(final, 'key_stats.tex')

if not CONVERT_TO_PDF:
    display(final)

In [5]:
# binarize categorical variables, drop NaNs and normalize and scale "age" between 0 and 1
data_binarized = pd.get_dummies(data).dropna(axis=0, how="any")
data_binarized["child"] = (data_binarized["age"] < 15).astype(int)
data_binarized["elderly"] = (data_binarized["age"] > 60).astype(int)
data_binarized["age"] = preprocessing.minmax_scale(preprocessing.scale(np.array(data_binarized["age"])))

if not CONVERT_TO_PDF:
    display(data_binarized.head(n=3))

In [6]:
# create arrays for a stan model
features = ["age",
            "child",
            "elderly",
            "pclass_1st", 
            "pclass_2nd", 
            "pclass_3rd", 
            "embarked_Cherbourg", 
            "embarked_Queenstown", 
            "embarked_Southampton", 
            "sex_female", 
            "sex_male"]

y = np.array(data_binarized["survived"])
X = np.array(data_binarized[features], dtype=np.dtype(float))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print("{0} samples in training set \n{1} samples in test set".format(y_train.size, y_test.size))

424 samples in training set 
209 samples in test set


### Model Training and Selection

In [7]:
def fit_model(prior_df, prior_s):
    model_data = dict(n=X_train.shape[0], 
                      d=X_train.shape[1], 
                      X=X_train, 
                      y=y_train, 
                      p_beta_df=prior_df, 
                      p_beta_scale=prior_s)
    fit = model.sampling(data=model_data, seed=1, control=dict(max_treedepth=15))
    return fit

In [8]:
prior_dfs = [4, 5, 6]
prior_scale = [1, 2, 4, 10, 20]
model = stan_utility.compile_model('logistic_regression.stan')

if RUN_MODEL_SELECTION:    
    fig, axs = plt.subplots(len(prior_dfs), len(prior_scale), figsize=(17, 14))
    axs = axs.ravel()
    p_effs = []
    loo_sums = []
    i = 0

    for df in tqdm(prior_dfs):
        for s in prior_scale:
            fit = fit_model(df, s)
            samples = fit.extract(permuted=True)

            # LOO CV
            loo, loos, ks = psis.psisloo(samples["log_lik"])
            loo_sums.append(loo)

            lppd = np.sum(np.log(np.sum(np.exp(samples["log_lik"]), axis=0)/4000))
            p_effs.append(lppd-loo)

            datapoints = np.arange(1, X_train.shape[0] + 1)
            axs[i].plot(datapoints, ks, 'o')
            axs[i].plot(datapoints, [0.7] * X_train.shape[0])
            axs[i].set_title("prior t({0}, {1}, {2}) k-values".format(df, 0, s))
            i += 1
            
    fig.savefig('k_values.png', bbox_inches='tight')
    with open("loo_sums.pkl", 'wb') as f:
        pickle.dump(loo_sums ,f, protocol=2)
    with open("p_effs.pkl", 'wb') as f:
        pickle.dump(p_effs ,f, protocol=2)

if not CONVERT_TO_PDF:
    img=image.imread('k_values.png')
    plt.figure(figsize = (50,50))
    plt.imshow(img)
    plt.show()

Using cached StanModel


In [9]:
if not RUN_MODEL_SELECTION:
    with open("loo_sums.pkl", 'rb') as f:
        loo_sums = pickle.load(f)
    with open("p_effs.pkl", 'rb') as f:
        p_effs = pickle.load(f)

psis_loo_results = pd.DataFrame()
psis_loo_results["p_eff"] = p_effs
psis_loo_results["PSIS-LOO"] = loo_sums
psis_loo_results["prior df"] = [i for i, _ in itertools.product(prior_dfs, prior_scale)]
psis_loo_results["prior scale"] = [j for _, j in itertools.product(prior_dfs, prior_scale)]
psis_loo_results_table = psis_loo_results.set_index(["prior df", "prior scale"], drop=True)
create_tex_table(psis_loo_results_table, 'psis_loo_results.tex')

if not CONVERT_TO_PDF:
    display(psis_loo_results)

In [10]:
# plot betas
largest_psis_loo_params = psis_loo_results[psis_loo_results["PSIS-LOO"] == 
                                           np.max(psis_loo_results["PSIS-LOO"])] 
fit = fit_model(int(largest_psis_loo_params["prior df"]), 
                int(largest_psis_loo_params["prior scale"]))
samples = fit.extract(permuted=True)

if not CONVERT_TO_PDF:
    m = 4
    n = int(math.ceil(len(features)/float(m)))
    fig, axs = plt.subplots(n, m, figsize=(15, 12))
    for i, (ax, feature) in enumerate(zip(axs.flat, features)):
        ax.hist(samples["beta"][:,i], bins=100)
        ax.set_title(feature)
    fig.savefig('betas.png', bbox_inches='tight')


### Convergence Diagnostic

In [11]:
filt = ['alpha'] + ['beta[{}]'.format(i+1) for i in range(len(features))]
fit_logistic_regression = create_fit_table(fit, 'fit_logistic_regression.tex', filter=filt)

if not CONVERT_TO_PDF:
    display(fit_logistic_regression)

### Sensitivity Analysis

### Predictive Performance Assessment

In [12]:
def logistic(x, beta, alpha):
    return (1+np.exp(-(alpha + np.dot(x, beta))))**(-1)

def get_y_preds(data, beta, alpha):
    y_preds = []
    for x in data:
        res = logistic(x, beta, alpha)
        y_preds.append(1 if res > 0.5 else 0)
    return y_preds
        
def check_accuracy(data, target, beta, alpha):
    ans_list = []
    for i in range(len(data)):
        res = logistic(data[i], beta, alpha)
        ans = 1 if res > 0.5 else 0
        ans_list.append(ans == target[i])

    return np.mean(ans_list)

def check_dummy_accuracy(target, res):
    ans_list = []
    for i in range(len(res)):
        ans_list.append(res[i] == target[i])
    return np.mean(ans_list)

mean_list = fit.summary()["summary"]
beta = mean_list[1:len(features)+1, 0]
alpha = mean_list[0, 0]

predictive_performance = pd.DataFrame()
predictive_performance["Dummy (All 1) training"] = [f1_score(y_train, [1] * len(y_train))]
predictive_performance["training"] = [f1_score(y_train, get_y_preds(X_train, beta, alpha))]
predictive_performance["Dummy (All 1) test"] = [f1_score(y_test, [1] * len(y_test))]
predictive_performance["test"] = [f1_score(y_test, get_y_preds(X_test, beta, alpha))]

create_tex_table(predictive_performance, 'f1_scores.tex')

if not CONVERT_TO_PDF:
    display(predictive_performance)

### my_utilities.py

### logistic_regression.stan